# Librerias necesarias

In [1]:
import pandas as pd
import os

# Importacion de funciones.
import import_ipynb
import funciones as f

importing Jupyter notebook from funciones.ipynb


# Extraccion

In [2]:
# Ruta de la carpeta que almacena todos los archivos CSV

dir_path = f"{os.path.dirname(os.getcwd())}/archivos"

In [3]:
# Se busca extraer los archivos .csv guardados en la carpeta archivos.

df_factors = pd.read_csv(f'{dir_path}/factors.csv')
df_intensity = pd.read_csv(f'{dir_path}/intensity.csv')
df_regional = pd.read_csv(f'{dir_path}/regional.csv')
df_date = pd.read_csv(f'{dir_path}/date.csv')
df_generation = pd.read_csv(f'{dir_path}/generation.csv')
df_regions = pd.read_csv(f'{dir_path}/regions.csv')


# Transformacion

### df_factors

In [4]:
# Se cambia el tipo de dato de la columna "fuels" a string.

df_factors = df_factors.rename(columns = {'Fuels':'fuels'})
df_factors['fuels'] = df_factors['fuels'].astype("string")


### df_intensity

In [5]:

# Cambiar a tipo de datos correctos.
df_intensity['from'] = pd.to_datetime(df_intensity['from'])
df_intensity['intensity.index'] = df_intensity['intensity.index'].astype('string')

# Eliminar columna to
df_intensity = df_intensity.drop(columns= ['to'])

# Crear columna año, mes, dia y hora
df_intensity['hour'] = df_intensity['from'].dt.time
df_intensity['month'] = df_intensity['from'].dt.month
df_intensity['day'] = df_intensity['from'].dt.day
df_intensity['year'] = df_intensity['from'].dt.year

# Filtrar los registros obtenidos por hora
df_intensity = df_intensity[df_intensity['hour'].apply(lambda x: x.minute == 0)]

# Reordenar ubicacion de columnas
df_intensity = df_intensity.reindex(columns=['from','year','month','day', 'hour', 'intensity.forecast', 'intensity.actual', 'intensity.index'])

#renombrar columna from a date
df_intensity = df_intensity.rename(columns = {
    'from':'col_date',
    'intensity.forecast': 'intensity_forecast',
    'intensity.actual':'intensity_actual',
    'intensity.index':'intensity_index'})

# Resetear indices
df_intensity.reset_index(drop=True, inplace= True)

# Cambiar tipo de dato
df_intensity['hour']= df_intensity['hour'].astype(str).str.split(':').str[0].astype(int)


In [6]:
# Modificar formato de fecha de la columna date, se busca eliminar el '+00:00' sin cambiar de tipo de dato.
df_intensity['col_date'] = pd.to_datetime(df_intensity['col_date'].dt.strftime('%Y-%m-%d %H:%M:%S'))

### df_regions

In [7]:
#Cambio de tipo de dato de la columna shortname a string.
df_regions['shortname'] = df_regions['shortname'].astype("string")

### df_date

In [8]:
# Modificar formato de fecha de la columna date, se busca eliminar el '+00:00' sin cambiar de tipo de dato.
df_date['date'] = pd.to_datetime(df_date['date']).dt.strftime('%Y-%m-%d %H:%M:%S')


# Cambiar tipo de datos de la columnas: year, month, day, hour y cod_date.
# Reordenar el orden de las columnas.
# Se vuelve a convertir a la columna date en tipo de dato datetime, debido a que la funcion strftime retorna un tipo de dato string.
df_date['year']= df_date['year'].astype(int)
df_date['month']= df_date['month'].astype(int)
df_date['day']= df_date['day'].astype(int)
df_date['hour']= df_date['hour'].astype(str).str.split(':').str[0].astype(int)
df_date['cod_date'] = df_date['cod_date'].astype("string")
df_date['date'] = pd.to_datetime(df_date['date'])

#Renombrar columna
df_date.rename(columns = {'date': 'col_date'}, inplace= True)

df_date = df_date.reindex(
    columns=['col_date','year','month','day', 'hour', 'cod_date']
    )

### df_generation

In [9]:
#Conversion de tipo de datos.
df_generation['fuel'] = df_generation['fuel'].astype("string")
df_generation['cod_date'] = df_generation['cod_date'].astype("string")
df_generation['regionid'] = df_generation['regionid'].astype(int)

#Reordenar columnas.
df_generation = df_generation.reindex(
    columns=['regionid','cod_date','fuel','perc']
    )

#Convertir las primeras letras de los combustibles de la columna fuel en mayusculas.
df_generation ['fuel'] = df_generation['fuel'].str.capitalize()

#renombrar columna regionid a region_id
df_generation = df_generation.rename(columns = {'regionid':'region_id'})

### df_regional

In [10]:
# Eliminar columnas
df_regional.drop(columns = ['dnoregion', 'shortname', 'generationmix'], inplace = True)

# Reordenar columnas.
df_regional = df_regional.reindex(
    columns=['region_id','cod_date','intensity.forecast','intensity.index']
    )

# Cambiar tipo de datos.
df_regional['cod_date'] = df_regional['cod_date'].astype("string")
df_regional['region_id'] = df_regional['region_id'].astype(int)
df_regional['intensity.forecast'] = df_regional['intensity.forecast'].astype(int)
df_regional['intensity.index'] = df_regional['intensity.index'].astype('string')

# Renombrar columnas
df_regional.rename(columns={'intensity.forecast':'intensity_forecast',
                            'intensity.index':'intensity_index'}, inplace=True)

### Localizar valores nulos en todos los dataframes

In [11]:
#Localizar valores nulos en todos los dataframes

lista_dfs = [df_intensity, df_factors, df_date, df_generation, df_regional, df_regions]
nombre_dfs = ['df_intensity', 'df_factors', 'df_date', 'df_generation', 'df_regional', 'df_regions']
i=0
for df in lista_dfs:
  suma = df.isnull().sum()
  print(f'DATAFRAME: {nombre_dfs[i]}')
  print('')
  print(suma)
  print('-------------------')
  print('')
  i=i+1


DATAFRAME: df_intensity

col_date              0
year                  0
month                 0
day                   0
hour                  0
intensity_forecast    0
intensity_actual      0
intensity_index       0
dtype: int64
-------------------

DATAFRAME: df_factors

fuels    0
value    0
dtype: int64
-------------------

DATAFRAME: df_date

col_date    0
year        0
month       0
day         0
hour        0
cod_date    0
dtype: int64
-------------------

DATAFRAME: df_generation

region_id    0
cod_date     0
fuel         0
perc         0
dtype: int64
-------------------

DATAFRAME: df_regional

region_id             0
cod_date              0
intensity_forecast    0
intensity_index       0
dtype: int64
-------------------

DATAFRAME: df_regions

region_id    0
shortname    0
dtype: int64
-------------------



## Establecer relacion entre dataframes

In [12]:
#Relacion entre DF: generation, regions y date.
#Se busca crear un DF que almacene los percentiles de los fines de semana (sabados y domingos.)

df_perc_wknd= pd.merge(pd.merge(df_generation,df_regions, on= 'region_id', how = 'inner'),
                       df_date, on= 'cod_date', how = 'inner')

#Crear columna dias de semana
df_perc_wknd['day_name'] = df_perc_wknd['col_date'].dt.day_name().astype("string")

# Obtener registros de percentiles de los fin de semanas.
dias = ['Saturday', 'Sunday']
filtro = df_perc_wknd['day_name'].isin(dias)
df_perc_wknd = df_perc_wknd[filtro].reset_index(drop= True)

#Reordenar columnas
df_perc_wknd = df_perc_wknd.reindex(
    columns=['region_id','cod_date', 'year','month','day', 'hour', 'day_name', 'fuel', 'perc']
    )

# Carga

In [13]:
# Se realiza la carga de datos a una base de datos postgresql. Se hace uso de la funcion connect_to_postgres para realizar la conexion.
conn_path = f"{os.path.dirname(os.getcwd())}/credenciales/config.ini"
engine = f.connect_to_postgres(conn_path)

In [14]:
# Truncar tablas.
# El objetivo de truncar tablas es evitar la duplicacion de datos.
# Proximamente: crear un Procedimiento Almacenado que lo evite.

engine.execute('truncate table ba_regions')
engine.execute('truncate table ba_intensity_national')
engine.execute('truncate table ba_factors')
engine.execute('truncate table ba_time')
engine.execute('truncate table ba_regional')
engine.execute('truncate table ba_generation')
engine.execute('truncate table ba_perc_weekend')

In [15]:
df_regions.to_sql('ba_regions',
                  engine,
                  method="multi", chunksize=1000,
                  if_exists= 'append',
                  index = False)

In [16]:
df_intensity.to_sql('ba_intensity_national',
                  engine,
                  method="multi", chunksize=1000,
                  if_exists= 'append',
                  index = False)

In [17]:
df_factors.to_sql('ba_factors',
                  engine,
                  method="multi", chunksize=1000,
                  if_exists= 'append',
                  index = False)

In [18]:
df_date.to_sql('ba_time',
                  engine,
                  method="multi", chunksize=1000,
                  if_exists= 'append',
                  index = False)

In [19]:
df_regional.to_sql('ba_regional',
                  engine,
                  method="multi", chunksize=5000,
                  if_exists= 'append',
                  index = False)

In [20]:
df_generation.to_sql('ba_generation',
                  engine,
                  method="multi", chunksize=5000,
                  if_exists= 'append',
                  index = False)

In [21]:
df_perc_wknd.to_sql('ba_perc_weekend',
                  engine,
                  method="multi", chunksize=5000,
                  if_exists= 'append',
                  index = False)